In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

In [2]:
def get_model(vocab_size, emb_dim, max_len=256, n_attention_heads=6, emb_trainable=True) :
    inputs      = tf.keras.Input(shape=(max_len,), dtype='int32')
    inputs_mask = tf.keras.Input(shape=(max_len,), dtype='bool')
    
    embedding_layer = keras.layers.Embedding(vocab_size, emb_dim, input_length=max_len, mask_zero=True, trainable=emb_trainable)
    
    embedding  = embedding_layer(inputs)
    last       = embedding
    
    enc_conv1  = keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(last)
    last       = enc_conv1

    if n_attention_heads == 1 :
        attention = keras.layers.Attention(causal=False)(
            [
            last, 
            last
            ],
            mask=[ 
            inputs_mask, 
            inputs_mask
            ]
        )
        last  = attention
    elif n_attention_heads > 1 :
        attentions = []
        for _ in range(n_attention_heads) :
            attentions.append(
                keras.layers.Attention(causal=False)(
                    [
                        last, 
                        last
                    ],
                    mask=[ 
                        inputs_mask, 
                        inputs_mask
                    ]
                )
            )
        attention = keras.layers.Add()(attentions)
        attention = keras.layers.BatchNormalization()(attention)
        last  = attention
    else :
        pass
    
    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(last)
    last  = conv1
    
    conv2 = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(last)
    last  = conv2
    
    pool1 = keras.layers.MaxPooling1D(pool_size=2, padding='same')(last)
    last  = pool1
    
    drop1 = keras.layers.Dropout(0.05)(last)
    last  = drop1
    
    conv3 = keras.layers.Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(last)
    last  = conv3
    
    conv4 = keras.layers.Conv1D(filters=8,  kernel_size=3, activation='relu', padding='same')(last)
    last  = conv4
    
    pool2 = keras.layers.GlobalAveragePooling1D()(last)
    last  = pool2
    
    drop2 = keras.layers.Dropout(0.05)(last)
    last  = drop2

    outputs = keras.layers.Dense(1, activation='sigmoid')(last)

    model = keras.models.Model(inputs=[inputs, inputs_mask], outputs=outputs)

    model.summary()

    return model, embedding_layer

In [3]:
import pandas as pd
import json

def get_word_index(sentences) :
    all_tokens = []
    for txt in sentences :
        all_tokens += txt.split()

    tokens = pd.Series(all_tokens, range(len(all_tokens)), name="tokens")
    types = tokens.unique()
    word_index = {word : i for i, word in enumerate(["<pad>", "<unk>"] + list(types))}
    
    def decode_review(text):
        reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
        return ' '.join([reverse_word_index.get(i, '?') for i in text])
    
    return word_index, decode_review

def get_dataset(dataframe, word_index, max_len=256) :
    index_series, x_series, y_series = dataframe.index, dataframe["text"], dataframe["class"]

    index = list(index_series)

    x_list = [txt.split() for txt in list(x_series)]
    
    x_seq = []
    for tknlst in x_list :
        seq = []
        for tkn in tknlst :
            try :
                seq.append(word_index[tkn])
            except KeyError :
                seq.append(word_index["<unk>"])
        seq = (seq + [0] * (max_len - len(seq))) if (len(seq) < max_len) else (seq[ : max_len])
        x_seq.append(seq)
    
    y_int = [1 if lb >= .5 else 0 for lb in list(y_series)]

    x = np.array(x_seq, dtype=int) 
    mask = x != 0

    y = np.array(y_int, dtype=int) 

    return index, x, mask, y

In [4]:
train_dataframe = pd.read_csv("../resources/datasets/StanfordSentimentTreebank/split/SST2Processed2-train.csv", index_col=0)
word_index, decode_review = get_word_index(train_dataframe["text"])
ids_train, x_train, mask_train, y_train = get_dataset(train_dataframe, word_index)
len(ids_train), x_train.shape, mask_train.shape, y_train.shape

(8544, (8544, 256), (8544, 256), (8544,))

In [5]:
x_train[1], mask_train[1]

(array([ 2,  3, 32, 33, 34, 35,  3, 36, 35,  3, 37, 38,  5, 39, 40, 14, 18,
        41, 35, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 35, 53, 54, 55,
        56,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 

In [6]:
y_train[1]

1

In [7]:
decode_review(x_train[1])

"<start> the gorgeously elaborate continuation of the lord of the rings trilogy is so huge that a column of words can not adequately describe co writerdirector peter jackson's expanded vision of jrr tolkien's middle earth <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad

In [8]:
val_dataframe = pd.read_csv("../resources/datasets/StanfordSentimentTreebank/split/SST2Processed2-dev.csv", index_col=0)
ids_val, x_val, mask_val, y_val = get_dataset(val_dataframe, word_index)
len(ids_val), x_val.shape, mask_val.shape, y_val.shape

(1101, (1101, 256), (1101, 256), (1101,))

In [9]:
len(word_index)

16177

In [10]:
embed_dim = 768
embeddings_df = pd.read_csv(f"../resources/embeddings/StanfordSentimentTreebank/SST2Processed2-train_dim{embed_dim}.csv", index_col=0)
n_attention_heads = 6
emb_trainable = True
experimet_name = f"cnn1d_{n_attention_heads}ATTENTION_embed{embed_dim}"
checkpoint_dir = f"../resources/output/SST2Processed2/checkpoint_{experimet_name}.hdf5"

model, emb_layer = get_model(len(word_index), embed_dim, n_attention_heads=n_attention_heads, emb_trainable=emb_trainable)

TypeError: list.append() takes no keyword arguments

In [ ]:
emb_wgts = emb_layer.get_weights()
emb_wgts[0][0] = embeddings_df.iloc[0].values
emb_wgts[0][1 - embeddings_df.shape[0] : ] = embeddings_df.values[1:]
emb_layer.set_weights(emb_wgts)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit([x_train, mask_train],
                    y_train,
                    epochs=100,
                    batch_size=64,
                    validation_data=([x_val, mask_val], y_val),
                    callbacks=[
                        keras.callbacks.ModelCheckpoint(
                            checkpoint_dir,
                            save_best_only=True,
                            save_weights_only=False,
                            verbose=1
                        ),
                        keras.callbacks.EarlyStopping(
                            monitor='accuracy',
                            min_delta=.001,
                            patience=30
                        )
                    ],
                    verbose=1)

Epoch 1/100
134/134 [==============================] - ETA: 0s - loss: 0.6901 - accuracy: 0.5394
Epoch 1: val_loss improved from inf to 0.67735, saving model to ../resources/output/SST2Processed2\checkpoint_cnn1d_1ATTENTION_embed768.hdf5
134/134 [==============================] - 57s 418ms/step - loss: 0.6901 - accuracy: 0.5394 - val_loss: 0.6773 - val_accuracy: 0.6013
Epoch 2/100
134/134 [==============================] - ETA: 0s - loss: 0.6727 - accuracy: 0.5812
Epoch 2: val_loss improved from 0.67735 to 0.66144, saving model to ../resources/output/SST2Processed2\checkpoint_cnn1d_1ATTENTION_embed768.hdf5
134/134 [==============================] - 56s 419ms/step - loss: 0.6727 - accuracy: 0.5812 - val_loss: 0.6614 - val_accuracy: 0.6240
Epoch 3/100
134/134 [==============================] - ETA: 0s - loss: 0.6330 - accuracy: 0.6440
Epoch 3: val_loss improved from 0.66144 to 0.65606, saving model to ../resources/output/SST2Processed2\checkpoint_cnn1d_1ATTENTION_embed768.hdf5
134/134 [=

In [ ]:
model.load_weights(checkpoint_dir)

In [ ]:
test_dataframe = pd.read_csv("../resources/datasets/StanfordSentimentTreebank/split/SST2Processed2-test.csv", index_col=0)
ids_test, x_test, mask_test, y_test = get_dataset(test_dataframe, word_index)
len(ids_test), x_test.shape, mask_test.shape, y_test.shape

(2210, (2210, 256), (2210, 256), (2210,))

In [ ]:
preds = model.predict([x_test, mask_test])
preds.shape

70/70 [==============================] - 3s 42ms/step


(2210, 1)

In [ ]:
preds_df = pd.DataFrame(preds, index=ids_test, columns=["predictions"])
preds_df

,predictions
3,0.459471
4,0.489437
5,0.559897
6,0.605639
7,0.707029
...,...
11621,0.566334
11623,0.914556
11626,0.871059
11628,0.449026


In [ ]:
dataset_with_preds_df = pd.concat([test_dataframe, preds_df], axis=1)
dataset_with_preds_df

,text,phrase_id,class,predictions
3,<start> effective but too tepid biopic,13995,0.513890,0.459471
4,<start> if you sometimes like to go to the mov...,14123,0.736110,0.489437
5,<start> emerges as something rare an issue mov...,13999,0.861110,0.559897
6,<start> the film provides some great insight i...,14498,0.597220,0.605639
7,<start> offers that rare combination of entert...,14351,0.833330,0.707029
...,...,...,...,...
11621,<start> an imaginative comedythriller,13851,0.777780,0.566334
11623,<start> a rare beautiful film,18182,0.916670,0.914556
11626,<start> an hilarious romantic comedy,23211,0.888890,0.871059
11628,<start> never sinks into exploitation,26177,0.625000,0.449026


In [ ]:
hits, total = 0, 0
for _, row in dataset_with_preds_df.iterrows() :
    total += 1
    pred  = row["predictions"] >= .5
    label = row["class"]       >= .5
    if pred == label : 
        hits += 1
accuracy = hits / total
accuracy

0.6705882352941176

In [ ]:
hits, total = 0, 0
for _, row in dataset_with_preds_df.iterrows() :
    total += 1
    pred  = row["predictions"]
    label = row["class"]
    diff = label - pred
    hits += abs(diff)
avg_dev = hits / total
avg_dev

0.20405662796735688

In [ ]:
true_positives, true_negatives, false_positives, false_negatives = 0, 0, 0, 0
for _, row in dataset_with_preds_df.iterrows() :
    pred  = row["predictions"] >= .5
    label = row["class"]       >= .5
    if pred == label :
        if pred :
            true_positives  += 1
        else :
            true_negatives  += 1
    else :
        if pred :
            false_positives += 1
        else :
            false_negatives += 1
print(true_positives, true_negatives, false_positives, false_negatives)

766 716 383 345


In [ ]:
precision = true_positives / (true_positives + false_positives)
precision

0.6666666666666666

In [ ]:
recall = true_positives / (true_positives + false_negatives)
recall

0.6894689468946895

In [ ]:
f1 = (2 * precision * recall) / (precision + recall)
f1

0.6778761061946904

In [ ]:
eval_dict = {
    'model'     : checkpoint_dir,
    'accuracy'  : accuracy,
    'precision' : precision,
    'recall'    : recall,
    'f1-score'  : f1
}

In [ ]:
with open(f"{eval_dict['model']}-eval-dict.json", "w") as f :
    f.write(json.dumps(eval_dict))